## Jupyter Notebook: HTTP Request Classifier

### Note about the prompt

First, I just copied the text from the coding task. Later I added the names of the files and a sample of the dataset. As some parts of the code did not work properly (like parsing_http_request and load_data functions) after a few iterations and debugging, we got to the point where we made it work. Nonetheless, the Gemini first solution was a rough scatch in a good direction, but it had some major drawback that was making the code ineffective, which I commented in the notebook. Those should be further fixed using more prompts.

Below is the prompt result:

This notebook demonstrates training a classifier to distinguish between malicious and normal HTTP requests using a labeled dataset.

**Libraries**


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score

# Feel free to import other libraries as needed

The code below was added, after suppling Gemini with a small sample of the training dataset file. The Data loading part was then adjusted (after several adjustments to the parsing and loading data functions)

In [18]:
# Assuming data has space delimiter
delimiter = " "

def parse_http_request(lines):
  """
  Parses a single HTTP request line into a dictionary.

  Args:
      lines (list): A list of strings representing the HTTP request lines (including headers).

  Returns:
      dict: A dictionary containing request method, URL, protocol, and headers (optional).
  """
  try:
    parts = lines[0].split(" ")
    request = {"method": parts[0], "url": parts[1], "protocol": parts[2]}
  except IndexError:
    return {}
  headers = {}
  for line in lines[1:]:
      if line.strip():  # Check if line has content
          key, value = line.split(":", 1)
          headers[key.strip()] = value.strip()
      else:
          break  # Reached end of headers (or empty line)
  request["headers"] = headers if headers else None  # Set headers to None if empty
  return request


def load_data(filename):
  """
  Loads HTTP request data from a file.

  Args:
      filename (str): The filename of the data file.

  Returns:
      list: A list of dictionaries, each representing an HTTP request.
  """
  data = []
  current_request = []  # Create an empty list to store lines for a single request
  with open(filename, "r") as f:
    for line in f:
      # Check if line is empty (ignoring empty lines between requests)
      if not line.strip():
        if current_request:  # If current_request has lines, process it
          data.append(parse_http_request(current_request))
          current_request = []  # Reset for the next request
      else:
        current_request.append(line.strip())  # Add non-empty lines to current request
  # Process the last request if it exists
  if current_request:
    data.append(parse_http_request(current_request))
  return data


**Load Data**

In [17]:

# Load data using the custom function
normal_train_data = load_data("csic_dataset/normalTrafficTraining.txt")
normal_test_data = load_data("csic_dataset/normalTrafficTest.txt")
anomalous_test_data = load_data("csic_dataset/anomalousTrafficTest.txt")

# Combine normal and anomalous data for supervised learning
all_data = normal_train_data + normal_test_data + anomalous_test_data
all_data = pd.DataFrame(all_data)  # Convert to pandas dataframe for further processing



**Note: Gemini did not label the data, ie. it did not create a target column where normal data would be 0 and anomalous data would be 1. We definitely need to add this to make any sense of the training at all.**

In [20]:
## Gemini did not add any data exploration lines of code, so
# I'm adding this part just to see the form of the final dataframe
all_data.head()

,method,url,protocol,headers
0,GET,http://localhost:8080/tienda1/index.jsp,HTTP/1.1,{'User-Agent': 'Mozilla/5.0 (compatible; Konqu...
1,GET,http://localhost:8080/tienda1/publico/anadir.j...,HTTP/1.1,{'User-Agent': 'Mozilla/5.0 (compatible; Konqu...
2,POST,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,{'User-Agent': 'Mozilla/5.0 (compatible; Konqu...
3,NaN,NaN,NaN,NaN
4,GET,http://localhost:8080/tienda1/publico/autentic...,HTTP/1.1,{'User-Agent': 'Mozilla/5.0 (compatible; Konqu...


**Note: We see that there are some errors in processing the input dataset, as there are NaN rows (4th row above). We definitely need to remove those, but Gemini forgot to include that step.**

In [23]:
all_data.headers.loc[0]

{'User-Agent': 'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.8 (like Gecko)',
 'Pragma': 'no-cache',
 'Cache-control': 'no-cache',
 'Accept': 'text/xml,application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5',
 'Accept-Encoding': 'x-gzip, x-deflate, gzip, deflate',
 'Accept-Charset': 'utf-8, utf-8;q=0.5, *;q=0.5',
 'Accept-Language': 'en',
 'Host': 'localhost:8080',
 'Cookie': 'JSESSIONID=1F767F17239C9B670A39E9B10C3825F4',
 'Connection': 'close'}

In [28]:
all_data.headers.loc[12]

{'User-Agent': 'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.8 (like Gecko)',
 'Pragma': 'no-cache',
 'Cache-control': 'no-cache',
 'Accept': 'text/xml,application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5',
 'Accept-Encoding': 'x-gzip, x-deflate, gzip, deflate',
 'Accept-Charset': 'utf-8, utf-8;q=0.5, *;q=0.5',
 'Accept-Language': 'en',
 'Host': 'localhost:8080',
 'Cookie': 'JSESSIONID=EE6A06EE8174FC2B05432AA5A0FCE998',
 'Content-Type': 'application/x-www-form-urlencoded',
 'Connection': 'close',
 'Content-Length': '33'}

We see that the method above did not disect the headers part, as well the url column. Processing the two column is definitely needed for a better prediction.

### Adding missing code

Here, I added manually, all the things that were missing, since the rest of the code would not run at all if the data was not processed correctly.

First, I added the target column before concating the datasets.

In [36]:
# To correct this we will first add the target column before concating the dataframes
normal_train = pd.DataFrame(normal_train_data)
normal_train['target'] = 0
normal_test = pd.DataFrame(normal_test_data)
normal_test['target'] = 0
anomalous_test = pd.DataFrame(anomalous_test_data)
anomalous_test['target'] = 1

# Combine normal and anomalous data for supervised learning
all_data = pd.concat([normal_train,normal_test,anomalous_test])
all_data.tail()


,method,url,protocol,headers,target
35037,POST,http://localhost:8080/tienda1/miembros/editar.jsp,HTTP/1.1,{'User-Agent': 'Mozilla/5.0 (compatible; Konqu...,1
35038,NaN,NaN,NaN,NaN,1
35039,GET,http://localhost:8080/tienda1/miembros/editar....,HTTP/1.1,{'User-Agent': 'Mozilla/5.0 (compatible; Konqu...,1
35040,GET,http://localhost:8080/tienda1/imagenes/3.gif/,HTTP/1.1,{'User-Agent': 'Mozilla/5.0 (compatible; Konqu...,1
35041,GET,http://localhost:8080/tienda1/imagenes/nuestra...,HTTP/1.1,{'User-Agent': 'Mozilla/5.0 (compatible; Konqu...,1


In [38]:
# Then I removed NaN rows

all_data.dropna(inplace=True)


**Feature Engineering**


This step is crucial for classifier performance and needs to be tailored to your data. Nonetheless, Gemini did not suggest any code for this part. So, I asked it to give me some solutions for feature engineering. Here is the output of that query:

Here are some potential feature engineering solutions you can explore based on the parsed HTTP request data (dictionary format):

**Basic Features:**

* **Method:** Categorical feature indicating the HTTP method used (e.g., GET, POST, PUT, DELETE). You can encode this using one-hot encoding or label encoding.
* **URL Path:** Extract the path component from the URL. This can reveal the targeted resource on the server. You can further process this by:
    * Splitting the path into segments (e.g., "/products/123") and using them as features.
    * Identifying specific patterns or keywords in the path that might be indicative of certain functionalities.
* **Protocol Version:** Categorical feature indicating the HTTP protocol version (e.g., HTTP/1.1, HTTP/2).
* **Status Code (if available):** This might be present in the response data, but if included in your request logs, it can be a valuable indicator of success or error conditions.
* **Content Length (if available):** This might be present in the request headers, indicating the size of the request body (if any).

**Header-based Features:**

* **User-Agent:** Categorical feature indicating the browser or client making the request. You can potentially group similar user agents or identify known malicious ones.
* **Referer:** This header indicates the previous webpage that linked to the current request. It can be helpful in understanding the context of the request.
* **Content-Type (if available):** This indicates the format of the request body data (e.g., application/json, application/x-www-form-urlencoded).
* **Number of Headers:** This can be a simple feature indicating the complexity of the request.

**Advanced Features:**

* **Request Frequency:** Analyze access logs to calculate the frequency of specific request patterns over time. Deviations from normal patterns might indicate suspicious activity.
* **Time-based Features:** Extract features like hour of the day, day of the week, or even time zone from the timestamps (if available) in the logs. This can help identify unusual access patterns.
* **IP Address Features:** Analyze source IP addresses and identify geographically anomalous origins for requests. Geolocation databases can be used for this purpose.

**Remember:**

* The choice of features depends on the specific characteristics of your data and the type of anomalies you are trying to detect.
* Feature selection techniques like correlation analysis can help identify the most informative features for your model.
* Consider data anonymization for sensitive information like IP addresses before processing, especially if sharing data for model development or evaluation.

By exploring these feature engineering solutions and tailoring them to your data, you can create a more robust and informative feature set for training your HTTP request classifier.


**Train-Test Split**

In [19]:
X = all_data.drop("label", axis=1)  # Features
y = all_data["label"]  # Labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


KeyError: "['label'] not found in axis"

**Classifier Training (Isolation Forest)**

In [ ]:
# Isolation Forest is a good choice for anomaly detection
clf = IsolationForest(contamination=0.1)  # Adjust contamination parameter as needed
clf.fit(X_train)

**Prediction and Evaluation**

In [ ]:
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy on Test Set: {accuracy:.4f}")

# Additional evaluation metrics like precision, recall, F1 score can be included


**Further Exploration**

* Try different anomaly detection algorithms (e.g., Local Outlier Factor, One-Class SVM) and compare their performance.
* Experiment with feature selection techniques to identify the most informative features.
* Visualize the distribution of features for normal and anomalous data to understand patterns.

**Converting to Production**

* Save the trained model using libraries like pickle or joblib.
* Integrate the model into a web application or security system for real-time traffic classification.
* Continuously monitor model performance and retrain with new data to maintain accuracy.

**Note:**

* This is a basic example, and the specific implementation will depend on your data format and chosen features.
* Remember to adapt the feature engineering and classifier selection based on your specific dataset.